In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

In [2]:
df = pd.read_csv('./dataset/encoded_accidents.csv')

In [3]:
# X, y for training
X = df.drop('Severity', axis=1)
y = df['Severity']
y = [int(i)-1 for i in y]

In [4]:
X = X.astype('float32')
X = torch.tensor(X.values).float()
y = torch.tensor(y).long()

In [5]:
# train = 75 , test = 10 and validation = 15 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.6, random_state=42)

In [6]:
# Convert to batches
batch_size = 32

train = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

test = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

val = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)



In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:


class ANN(nn.Module):
    def __init__(self, input_features=5, hidden1=20, hidden2=20, output_features=4):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, output_features)
    
    def forward(self, x):
        x = torch.relu(self.f_connected1(x))
        x = torch.relu(self.f_connected2(x))
        x = self.out(x)
        return x
    
torch.manual_seed(20)


In [9]:
ann_model = ANN(input_features=X.shape[1], output_features=len(set(y))).to(device)

In [10]:
ann_model.parameters

<bound method Module.parameters of ANN(
  (f_connected1): Linear(in_features=21, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=7494396, bias=True)
)>

In [11]:
def acc_fn(y_pred, y_test):
    y_pred = torch.argmax(y_pred, 1)
    return torch.sum(y_pred == y_test).item()

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(ann_model.parameters(), lr=0.01)

epochs = 500
final_losses = []
final_acc = []
for i in range(epochs):
    i += 1
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        y_pred = ann_model(inputs)
        loss = loss_fn(y_pred, labels)
        acc = acc_fn(y_pred, labels)
        optim.zero_grad()
        loss.backward()
        optim.step()
    final_losses.append(loss.item())
    final_acc.append(acc/len(X_val))
    if i%100 == 0:
        with torch.no_grad():
            acc = 0
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                y_pred = ann_model(inputs)
    print(f'Epoch {i} Loss: {loss.item()} Accuracy: {acc/len(X_val)}')